## Deliverable 2. Create a Customer Travel Destinations Map.

In [5]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
#import gmaps.datasets

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

ModuleNotFoundError: No module named 'config'

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Provideniya,RU,64.3833,-173.3000,53.71,58,40,13.42,scattered clouds
1,1,Ribeira Grande,PT,38.5167,-28.7000,74.08,78,3,11.81,clear sky
2,2,Broome,US,42.2506,-75.8330,78.71,75,20,10.36,few clouds
3,3,Bubaque,GW,11.2833,-15.8333,79.41,83,65,13.02,light rain
4,4,Punta Arenas,CL,-53.1500,-70.9167,37.51,81,100,10.36,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Broome,US,42.2506,-75.8330,78.71,75,20,10.36,few clouds
3,3,Bubaque,GW,11.2833,-15.8333,79.41,83,65,13.02,light rain
8,8,Tutoia,BR,-2.7619,-42.2744,78.37,80,20,11.97,few clouds
11,11,Brookhaven,US,31.5791,-90.4407,75.24,82,48,5.57,moderate rain
13,13,Hithadhoo,MV,-0.6000,73.0833,81.64,66,97,1.72,overcast clouds
14,14,Alofi,NU,-19.0595,-169.9187,78.69,69,75,12.66,broken clouds
16,16,Hilo,US,19.7297,-155.0900,89.28,79,75,5.75,broken clouds
21,21,Brewster,US,41.3973,-73.6171,84.54,82,0,9.22,moderate rain
27,27,Gat,IL,31.6100,34.7642,75.88,70,0,2.26,clear sky
28,28,Socorro,US,31.6546,-106.3033,86.02,46,75,13.80,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
#n/a

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Broome,US,78.71,few clouds,42.2506,-75.8330,
3,Bubaque,GW,79.41,light rain,11.2833,-15.8333,
8,Tutoia,BR,78.37,few clouds,-2.7619,-42.2744,
11,Brookhaven,US,75.24,moderate rain,31.5791,-90.4407,
13,Hithadhoo,MV,81.64,overcast clouds,-0.6000,73.0833,
14,Alofi,NU,78.69,broken clouds,-19.0595,-169.9187,
16,Hilo,US,89.28,broken clouds,19.7297,-155.0900,
21,Brewster,US,84.54,moderate rain,41.3973,-73.6171,
27,Gat,IL,75.88,clear sky,31.6100,34.7642,
28,Socorro,US,86.02,broken clouds,31.6546,-106.3033,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != ""]
clean_hotel_df.count()


City                   234
Country                234
Max Temp               234
Current Description    234
Lat                    234
Lng                    234
Hotel Name             234
dtype: int64

In [10]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [3]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

NameError: name 'gmaps' is not defined